# Overview

We will load the model and infer data from it.

In [1]:
import json

# Read config.json
with open('../config.json', 'r') as config_file:
    config = json.load(config_file)

# Hyperparameters

In [2]:
# Tokenize and pad the text data
max_len = 100  # Maximum length of input sequences

# Load Dependencies

In [3]:
import tensorflow as tf
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences

2024-08-09 19:53:50.371469: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-09 19:53:50.387904: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-09 19:53:50.393030: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-09 19:53:50.406749: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-09 19:53:51.523990: W tensorflow/compiler/tf2

# Load Model and Objects



In [4]:
# Load the trained model
model = tf.keras.models.load_model(f"{config['save_directory']}/models/{config['model_version']}-labeler_model.keras")

# Load other objects
with open(f"{config['save_directory']}/objects/{config['model_version']}-labeler_objects.pkl", 'rb') as f:
    tokenizer, label_encoder = pickle.load(f)

# Prediction

In [16]:
# Example message to predict
new_message = ["hard to track down the exact specs but I think its NVIDIA 940MX, I don't really recognize it though sorry"]

# Preprocess the new message
new_message_seq = tokenizer.texts_to_sequences(new_message)
new_message_padded = pad_sequences(new_message_seq, maxlen=max_len)

# Predict the class probabilities
class_probabilities = model.predict(new_message_padded)

# Sort the class probabilities in descending order along with their corresponding classes
sorted_probabilities = sorted(zip(label_encoder.classes_, class_probabilities[0]), key=lambda x: x[1], reverse=True)

# Set a threshold to filter out near-zero probabilities
threshold = 0.01

# Print the sorted class probabilities above the threshold
for reason, probability in sorted_probabilities:
    if probability > threshold:
        print(f"{reason}: {probability:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Politics not allowed outside of references to the market.: 0.9592
Clean: 0.0350
